In [3]:
from dotenv import load_dotenv
import os

load_dotenv(override=True) # Load environment variables from .env file, override any existing variables

# Making a Langchain Embeddings Object using Nomic

from langchain_nomic import NomicEmbeddings

embeddings = NomicEmbeddings(model="nomic-embed-text-v1.5")

# Making a Pinecone Vector Store Object

from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "nlp-module"  # change if desired
index = pc. Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# Making a Retriever Object (Allows you to find similar documents in your Pinecone index, given a query)

retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 10, "score_threshold": 0.5},
)

# Making a ChatGroq Object (This is the LLM model that will generate responses)

from langchain_groq import ChatGroq
llm = ChatGroq(model="llama3-8b-8192", stop_sequences= None, temperature=0)

# Function to format the retrieved documents, gotten from the retriever

def format_docs(docs):
    print("docs:", docs)
    print()
    return "\n\n".join(doc.page_content for doc in docs)


# Making a custon prompt which had two variables, "context" and question

# Note:This prompt_template expects a dictionary/JSON with the keys "context" and "question" as input

from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["context", "question"],
            template=( # The constructed prompt from the variables
                "You are an assistant for question-answering tasks. Use the following "
                "pieces of retrieved context to answer the question. If you don't know "
                "the answer, just say that you don't know. Use three sentences maximum "
                "and keep the answer concise.\n\n"
                
                "Question: {question}\n"
                "Context: {context}\n"
                "Answer:"
            )
            
        )
    )
])

# A simple function that logs the input and returns it

def logger(input):
    print(input)
    return input


# A chain with the modified prompt

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# The chain simply looks likes this:
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt_template
    | llm
    | StrOutputParser()
)

# respnse = rag_chain.invoke("Tell me about the paper: Attention is all you Need")

In [9]:
from IPython.display import Markdown, display

respnse = rag_chain.invoke("when is the Arabic course taught? tell me all the people that teach it")

display(Markdown(respnse))

No relevant docs were retrieved using the relevance score threshold 0.5


docs: []



Based on the provided context, I don't have any information about the Arabic course being taught. Therefore, I cannot provide an answer to the question.

If you provide the context, I'll be happy to help you with the question.

In [15]:
from dotenv import load_dotenv
import os

load_dotenv(override=True) # Load environment variables from .env file, override any existing variables

# Making a Langchain Embeddings Object using Nomic

from langchain_nomic import NomicEmbeddings

embeddings = NomicEmbeddings(model="nomic-embed-text-v1.5")

# Making a Pinecone Vector Store Object

from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "musab-bilal-rag"  # change if desired
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings, namespace="prog-ann")

# Making a Retriever Object (Allows you to find similar documents in your Pinecone index, given a query)

retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 5, "score_threshold": 0.5},
)

# Making a ChatGroq Object (This is the LLM model that will generate responses)

from langchain_groq import ChatGroq
llm = ChatGroq(model="llama3-8b-8192", stop_sequences= None, temperature=0)

# Function to format the retrieved documents, gotten from the retriever

def format_docs(docs):
    print("docs:", docs)
    print()
    return "\n\n".join(doc.page_content for doc in docs)


# Making a custon prompt which had two variables, "context" and question

# Note:This prompt_template expects a dictionary/JSON with the keys "context" and "question" as input

from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["context", "question"],
            template=( # The constructed prompt from the variables
                "You are an assistant for question-answering tasks. Use the following "
                "pieces of retrieved context to answer the question. If you don't know "
                "the answer, just say that you don't know.\n\n"
                "Keep the answer concise but answer completely.\n\n"
                "Give the exact line of context if you are asked for some sort of justification for your response.\n\n"
                
                "Question: {question}\n"
                "Context: {context}\n"
                "Answer:"
            )
            
        )
    )
])

# A simple function that logs the input and returns it

def logger(input):
    # print(input)
    return input


# A chain with the modified prompt

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# The chain simply looks likes this:
rag_chain = (
    {
        "context": retriever | format_docs | logger,
        "question": RunnablePassthrough()
    }
    | prompt_template
    | llm
    | StrOutputParser()
)

# respnse = rag_chain.invoke("Tell me about the paper: Attention is all you Need")

In [16]:
from IPython.display import Markdown, display

respnse = rag_chain.invoke("What programs are offered by the School of Business Studies at IBA?")

display(Markdown(respnse))

docs: [Document(id='txt81', metadata={}, page_content='53\n---\n PROGRAM\n ANNOUNCEMENT 2024-25 25\n\n Activities Studio\nMuHL\n---\n 26 PROGRAM\n ANNOUNCEMENT 2024-25\n\nActivities Studio\n aka\n UGRAD-Pakistan\n ashnaton DC\n BA\n---\nSchool of Business Studies\n---\n PROGRAM\n ANNOUNCEMENT 2024-25 28\n\nSchool of Business Studies (SBS)\nThe Institute of Business Administration, (IBA) is one of the most prestigious higher education institutions located in the cit y of Karachi, the ﬁnancial hub of Pakistan. The School of Business Studies (SBS) is the largest and\nﬂagship School in IBA. SBS is also a leading business school within Pakistan with a state-of-the-art campus that follows the be st international standards and offers faculty and students a unique learning and teaching\nexperience along with well-equipped spaces conducive for teamwork and creativity.'), Document(id='txt1', metadata={}, page_content='IBA Institute of\n Business Administr ation\n Karachi\nLeadership and Ideas fo

The School of Business Studies at IBA offers the following programs:

* BBA (Bachelor of Business Administration)
* BS (Accounting and Finance)
* MBA (Master of Business Administration)
* EMBA (Executive Master of Business Administration)
* MS (Master of Science) in various specializations, including:
	+ Finance
	+ Islamic Banking and Finance
	+ Management
	+ Marketing
* PhD (Doctor of Philosophy) in various specializations, including:
	+ Computer Science
	+ Economics
	+ Mathematics